# Initialization

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
plt.rc('font', size=16)

import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# Enable/disable test mode
test_mode = True

## Import and visualize the dataset

In [ ]:
# Set the path to the dataset
dataset = pd.read_csv('../input/homework2/Training.csv')
#dataset = pd.read_csv('../input/Training.csv')
print(dataset.shape) 
dataset.head().transpose()

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()
inspect_dataframe(dataset, dataset.columns)

# Data preprocessing

In [ ]:
if(test_mode):
    test_size = 10000
    X_train_raw = dataset.iloc[:-test_size]
    X_test_raw = dataset.iloc[-test_size:]
else:
    X_train_raw = dataset

# y_train_raw = y.iloc[:-test_size]
# Normalize both features and labels
X_min = X_train_raw.min()
X_max = X_train_raw.max()

X_train_raw = (X_train_raw-X_min)/(X_max-X_min)

if(test_mode):
    # y_test_raw = y.iloc[-test_size:]
    X_test_raw = (X_test_raw-X_min)/(X_max-X_min)



## Extracting sequences from the dataset

In [ ]:
def build_sequences(df, target_labels, window, stride, telescope):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float32')#float32
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float32')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
target_labels = dataset.columns
window = 600
stride = 10
telescope = 864

In [ ]:
future = dataset[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0)
future.shape

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
print(X_train.shape, y_train.shape)

if(test_mode):
    X_test, y_test = build_sequences(X_test_raw, target_labels, window, stride, telescope)
    print(X_test.shape, y_test.shape)

In [ ]:
def inspect_multivariate(X, y, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

In [ ]:
inspect_multivariate(X_train, y_train, target_labels, telescope)

# Train the model

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
print(y_train.shape)

batch_size = 128
epochs = 200

model_name = 'BiLSTM'

In [ ]:
def build_model(input_shape ,output_shape):
    #create model
    model = tfk.Sequential()
    model.add(tfkl.Bidirectional(tfkl.LSTM(units=256, return_sequences=True), input_shape=input_shape))
    model.add(tfkl.Dropout(0.2))

    model.add(tfkl.Bidirectional(tfkl.LSTM(units=256, return_sequences=True)))
    model.add(tfkl.Dropout(0.2))

    model.add(tfkl.Bidirectional(tfkl.LSTM(units=256)))
    model.add(tfkl.Dropout(0.2))

    model.add(tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu'))
    model.add(tfkl.Reshape(output_shape))
    model.add(tfkl.Conv1D(output_shape[-1], 1, padding='same'))

    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])


    return model


In [ ]:
model = build_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.2,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
exp_dir = ''
model.save(os.path.join(exp_dir, model_name))

# Analyze the results

In [ ]:
# Plot loss, accuracy and learning rate
best_epoch = np.argmin(history['val_loss'])

plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
# Predict the test set 
if(test_mode):
    predictions = model.predict(X_test)
    print(predictions.shape)

    mean_squared_error = tfk.metrics.mse(y_test.flatten(),predictions.flatten())
    mean_absolute_error = tfk.metrics.mae(y_test.flatten(),predictions.flatten())
    print(mean_squared_error, mean_absolute_error)

In [ ]:
def inspect_multivariate_prediction(X, y, pred, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].plot(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].plot(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), pred[idx,:,i], color='green')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

In [ ]:
if(test_mode):
    inspect_multivariate_prediction(X_test, y_test, predictions, target_labels, telescope)

In [ ]:
if(test_mode):
    maes = []
    for i in range(predictions.shape[1]):
        ft_maes = []
        for j in range(predictions.shape[2]):
            ft_maes.append(np.mean(np.abs(y_test[:,i,j]-predictions[:,i,j]), axis=0))
        ft_maes = np.array(ft_maes)
        maes.append(ft_maes)
    maes = np.array(maes)

In [ ]:
if(test_mode):
    future_predictions = model.predict(future)
    print(future_predictions.shape)

In [ ]:
if(test_mode):
    figs, axs = plt.subplots(len(target_labels), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(target_labels):
        axs[i].plot(np.arange(len(future[0,:,i])), future[0,:,i])
        axs[i].plot(np.arange(len(future[0,:,i]), len(future[0,:,i])+telescope), future_predictions[0,:,i], color='orange')
        axs[i].fill_between(
            np.arange(len(future[0,:,i]), len(future[0,:,i])+telescope), 
            future_predictions[0,:,i]+maes[:,i], 
            future_predictions[0,:,i]-maes[:,i], 
            color='orange', alpha=.3)
        axs[i].set_title(col)
        # axs[i].set_ylim(0,1)
    plt.show()